In [1]:
import os
import pickle
import numpy as np
from tqdm.notebook import tqdm

from matplotlib import pyplot as plt
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.utils import to_categorical, plot_model
from tensorflow.keras.layers import Input, Dense, LSTM, Embedding, Dropout, add

In [13]:
import pandas as pd
import tensorflow as tf

In [104]:
# WORKING_DIR = '/content/drive/MyDrive/bike'
# IMAGE_DIR = '/content/drive/MyDrive/bike/images'

In [12]:
data = pd.read_excel("qa3.xlsx")

In [10]:
data['image_id'] = data['image_id'].apply(lambda img: os.path.join('images', img+'.jpeg'))
data.head(2)

,question,answer,image_id
0,is the person wearing helmet?,yes,images\image1.jpeg
1,is the person wearing helmet?,no,images\image2.jpeg


In [107]:
# WORKING_DIR = '/content/drive/MyDrive/pathvqa'
# IMAGE_DIR = '/content/drive/MyDrive/pathvqa/images'

In [111]:
data.head(2)

,question,answer,image_id
0,is the person wearing helmet?,yes,images\image1.jpeg
1,is the person wearing helmet?,no,images\image2.jpeg


In [112]:
data['answer']=data['answer'].astype(str)

In [113]:
x=data.drop(columns=['answer'])
y=data['answer']

In [114]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.3,random_state=42)

In [117]:
# !pip install keras_preprocessing

In [118]:
# from keras_preprocessing.image import ImageDataGenerator

In [119]:
max_length = data['question'].str.len().max()
print(max_length)

29


In [120]:
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.preprocessing.image import load_img,img_to_array

In [121]:
def textokenizer(text):
  max_length = 24
  tok = Tokenizer()
  tok.fit_on_texts(x_train['question'])
  vocab_size = len(tok.word_index) + 1
  #print('Total unique words in the x_train',vocab_size)
  encoded_text = tok.texts_to_sequences(text)
  padded_text = pad_sequences(encoded_text, maxlen=max_length)  #padding zeros at the begining of each question so that each sequence will have same length
  #print(padded_text.shape)
  return padded_text, tok

In [122]:
#Each token is represented using 300-dim vector using pre-trained GloVe representation.
with open( 'glove_vector.pkl', 'rb') as f:
    model = pickle.load(f)
    glove_words =  set(model.keys())

# for train
_,tok = textokenizer(x_train['question'])
vocab_size = len(tok.word_index) + 1
embedding_matrix_train = np.zeros((vocab_size, 300))
for word, i in tok.word_index.items():
    if word in glove_words:
        embedding_vector = model[word]
        embedding_matrix_train[i] = embedding_vector

print(vocab_size)

6


In [123]:
##print("embedding matrix shape",embedding_matrix_train.shape)

In [124]:
# with open( 'embedding_matrix_train.pkl'), 'wb') as f:
#     pickle.dump(embedding_matrix_train, f)

In [125]:
# with open(os.path.join(WORKING_DIR, 'embedding_matrix_train.pkl'), 'rb') as f:
#     embedding_matrix_train = pickle.load(f)

In [126]:
def optokens(classes):
  from sklearn.preprocessing import OneHotEncoder
  ohe=OneHotEncoder(handle_unknown='ignore')
  ohe.fit(y_train.values.reshape(-1,1))
  optoken=ohe.transform(classes.values.reshape(-1,1)).toarray()
  return optoken, ohe

In [127]:
_,ohe=optokens(y_train)


In [128]:
pickle.dump(ohe,open('ohe.pkl','wb'))

In [129]:
# model = VGG16(weights='imagenet')
# feature_model = Model(inputs=model.inputs, outputs=model.layers[-2].output)

In [14]:
import tensorflow as tf

In [131]:
def parse_function(filename):
    image_string = tf.io.read_file(filename)
    #Don't use tf.image.decode_image, or the output shape will be undefined
    image = tf.image.decode_png(image_string, channels=3)
    image = image/255
    #This will convert to float values in [0, 1]
    image = tf.image.convert_image_dtype(image, tf.float32)
    image = tf.image.resize(image, [224, 224])
    image.set_shape((224, 224, 3))
    return  image, filename

In [132]:
# img_fl=sorted(set(data['image_id'].tolist()))
# img_data_tr=tf.data.Dataset.from_tensor_slices(img_fl)
# img_data_tr=img_data_tr.map(parse_function,num_parallel_calls=tf.data.experimental.AUTOTUNE).batch(32).prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
# for img, path in tqdm(img_data_tr):
#   batch_features = feature_model(img)
#   #batch_features = tf.reshape(batch_features,(batch_features.shape[0], -1, batch_features.shape[3]))

#   for bf, p in zip(batch_features, path):
#     path_of_feature = p.numpy().decode("utf-8")
#     np.save(path_of_feature, bf.numpy())

In [133]:
# import matplotlib.pyplot as plt
# from PIL import Image

# image_filenames = x_train['image_id']
# # Path to your images directory
# images_directory = IMAGE_DIR

# # Choose the first image ID
# first_image_id = image_filenames.iloc[0]

# # Loop through the data and find the first occurrence of the first image ID
# for image_filename, question, answer in zip(image_filenames, ques_train, answ_train):
#     if image_filename == first_image_id:
#         # Construct the full path to the image
#         image_path = image_filename

#         # Load and display the image
#         image = Image.open(image_path)
#         plt.imshow(image)
#         plt.title("Image Filename: {}\nQuestion: {}\nAnswer: {}".format(image_filename, question, answer))
#         plt.axis('off')
#         plt.show()
#         break  # Break the loop after displaying the first image


In [134]:
ques_train = x_train['question']
answ_train = y_train

In [135]:
ques_train , _ =textokenizer(x_train['question'])
answ_train , _ =optokens(y_train)

In [136]:
# print(y_train)

In [137]:
print(ques_train.shape)
print(answ_train.shape)

(147, 24)
(147, 2)


In [138]:
ques_train=ques_train.tolist()
answ_train=answ_train.tolist()

In [139]:
# print(ques_train[0])

In [140]:
# print(answ_train[0])

In [141]:
def map_func(img_name, question):
    img_tensor = np.load(str(img_name) + '.npy')  # Ensure img_name is treated as a string
    # print(img_tensor)
    return img_tensor, question

In [142]:
# from PIL import Image

In [143]:
image_filenames = x_train['image_id']

preprocessed_data = [(map_func(image_filenames, question), answer) for image_filenames, question, answer in zip(image_filenames, ques_train, answ_train)]

# Extract image features and tokenized questions from preprocessed data
image_features = np.array([item[0][0] for item in preprocessed_data])
tokenized_questions = np.array([item[0][1] for item in preprocessed_data])
answers = np.array([item[1] for item in preprocessed_data])

In [144]:
#image_features=np.resize(image_features,(224,224,3))

In [145]:
# print(preprocessed_data[0])

In [146]:
print(len(image_features))
print(len(tokenized_questions))
print(len(answers))

147
147
147


In [147]:
# print(image_features[0])
# print(tokenized_questions[0])
# print(answers[0])

In [148]:
# print(tokenized_questions[1])

In [149]:
tokenized_questions = tf.cast(tokenized_questions, tf.int64)
x_dataset = tf.data.Dataset.from_tensor_slices((image_features, tokenized_questions))
# answers = tf.cast(answers, tf.int64)
# Combine answers into a separate dataset
y_dataset = tf.data.Dataset.from_tensor_slices(answers)

# Combine x and y datasets into a single dataset
train_dataset = tf.data.Dataset.zip((x_dataset, y_dataset))

print(len(train_dataset))

# Batch and prefetch the dataset
batch_size = 12
train_dataset = train_dataset.batch(batch_size).prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

# Print dataset information
print(train_dataset)

147
<_PrefetchDataset element_spec=((TensorSpec(shape=(None, 4096), dtype=tf.float32, name=None), TensorSpec(shape=(None, 24), dtype=tf.int64, name=None)), TensorSpec(shape=(None, 2), dtype=tf.float64, name=None))>


In [150]:
# import tensorflow as tf

# # Convert PrefetchDataset to an iterable
# iter_train_dataset = iter(train_dataset)

# # Get the first element
# first_element = next(iter_train_dataset)
# second_element = next(iter_train_dataset)


# # Print the first element
# print(first_element)

In [151]:
tf.data.experimental.save(train_dataset,  'train_dataset')

In [152]:
train_dataset = tf.data.experimental.load('train_dataset', element_spec = ((tf.TensorSpec(shape=(None, 4096), dtype=tf.float32),
                 tf.TensorSpec(shape=(None, 24), dtype=tf.int64)),
                tf.TensorSpec(shape=(None, 2), dtype=tf.float64)))

In [153]:
ques_test,_=textokenizer(x_test['question'])
answ_test,_=optokens(y_test)

In [154]:
# print(ques_test[2])

In [155]:
ques_test=ques_test.tolist()
answ_test=answ_test.tolist()

In [156]:
image_filenames = x_test['image_id']

# print(image_filenames[1])
# Preprocess images and questions using map_func
preprocessed_data = [(map_func(image_filenames, question), answer) for image_filenames, question, answer in zip(image_filenames, ques_test, answ_test)]

# Extract image features and tokenized questions from preprocessed data
image_features = np.array([item[0][0] for item in preprocessed_data])
tokenized_questions = np.array([item[0][1] for item in preprocessed_data])
answers = np.array([item[1] for item in preprocessed_data])

In [157]:
print(len(image_features))
print(len(tokenized_questions))
print(len(answers))

63
63
63


In [158]:
tokenized_questions = tf.cast(tokenized_questions, tf.int64)
xo_dataset = tf.data.Dataset.from_tensor_slices((image_features, tokenized_questions))

# answers = tf.cast(answers, tf.int64)
# Combine answers into a separate dataset
yo_dataset = tf.data.Dataset.from_tensor_slices(answers)

# Combine x and y datasets into a single dataset
test_dataset = tf.data.Dataset.zip((xo_dataset, yo_dataset))

# Batch and prefetch the dataset
batch_size = 12
test_dataset = test_dataset.batch(batch_size).prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

# Print dataset information
print(test_dataset)

<_PrefetchDataset element_spec=((TensorSpec(shape=(None, 4096), dtype=tf.float32, name=None), TensorSpec(shape=(None, 24), dtype=tf.int64, name=None)), TensorSpec(shape=(None, 2), dtype=tf.float64, name=None))>


In [159]:
tf.data.experimental.save(test_dataset, 'test_dataset')

In [160]:
test_dataset = tf.data.experimental.load( 'test_dataset', element_spec = ((tf.TensorSpec(shape=(None, 4096), dtype=tf.float32),
                 tf.TensorSpec(shape=(None, 24), dtype=tf.int64)),
                tf.TensorSpec(shape=(None, 2), dtype=tf.float64)))

In [161]:
# from tensorflow.keras import initializers
# from tensorflow.keras.regularizers import l2
# from tensorflow.keras.layers import Input, Dense, Embedding, LSTM, Flatten, BatchNormalization, Dropout, multiply

In [162]:
#IMAGE MODEL
#im_input = Input(shape=(4096,), name = "im_input")
#flat = Flatten()(im_input)
#image_model1=Dense(50,kernel_initializer=initializers.he_normal(seed=42))(flat)
#image_model=Model(inputs=im_input,outputs=image_model1)
#image_model.summary()

In [15]:
import tensorflow as tf
import keras

In [164]:
# from keras.layers import Conv1D,MaxPool1D

In [165]:
# #IMAGE MODEL
# im_input = Input(shape=(4096,1), name = "im_input")
# #flat = Flatten()(im_input)
# layer1= (Conv1D(input_shape=(4096,1),filters=64,kernel_size=(3),padding="same", activation="relu"))(im_input)
# layer2=(Conv1D(filters=64,kernel_size=(3),padding="same", activation="relu"))(layer1)
# layer3=(MaxPool1D(pool_size=(2),strides=(2)))(layer2)
# d1 = Dropout(0.2)(layer3)
# layer4=(Conv1D(filters=128, kernel_size=(3), padding="same", activation="relu"))(d1)
# layer5=(Conv1D(filters=128, kernel_size=(3), padding="same", activation="relu"))(layer4)
# layer6=(MaxPool1D(pool_size=(2),strides=(2))(layer5))
# layer7=(Conv1D(filters=256, kernel_size=(3), padding="same", activation="relu")(layer6))
# layer8=(MaxPool1D(pool_size=(2),strides=(2)))(layer7)
# flat = Flatten()(layer8)
# image_model1=Dense(50,kernel_initializer=initializers.he_normal(seed=42))(flat)
# image_model=Model(inputs=im_input,outputs=image_model1)
# image_model.summary()

In [166]:
# max_length=24
# vocab_size=embedding_matrix_train.shape[0]
# print(vocab_size)

In [167]:
# #QUESTION MODEL
# from tensorflow.keras.layers import LSTM
# ques_input = Input(shape=(max_length,), name = "ques_input")
# e1 = Embedding(vocab_size, 300, weights=[embedding_matrix_train], input_length=131, trainable=False)(ques_input)
# lstm1= LSTM(64,kernel_initializer=initializers.he_normal(seed=42),kernel_regularizer=l2(0.001),return_sequences=True)(e1)
# lstm2= LSTM(64,kernel_initializer=initializers.he_normal(seed=42),kernel_regularizer=l2(0.001),return_sequences=True)(lstm1)
# #l1= LeakyReLU(alpha = 0.3)(l1)
# f1= Flatten(name='flatten_2')(lstm2)
# question_model1=Dense(50,kernel_initializer=initializers.he_normal(seed=42))(f1)
# question_model = Model(inputs=ques_input, outputs=question_model1)
# question_model.summary()

In [168]:
# print(image_model.layers[-1].output)

In [169]:
# #COMBINING FEATURES AND MAKING FINAL MODEL FOR PREDICTION
# from tensorflow.keras.layers import multiply
# from keras import regularizers
# input_model=multiply([image_model.layers[-1].output,question_model.layers[-1].output])
# d1=BatchNormalization()(input_model)
# d1 = Dropout(0.2)(d1)
# d1=Dense(50,kernel_initializer=initializers.he_normal(seed=42),kernel_regularizer=regularizers.l2(0.01))(d1)
# d1 = Dropout(0.2)(d1)
# final_output = Dense(8, kernel_initializer=initializers.he_normal(seed=42),kernel_regularizer=regularizers.l2(0.01),activation='softmax')(d1)
# final_model = Model(inputs=[im_input,ques_input], outputs=final_output)
# print(final_model.summary())
# plot_model(final_model, show_shapes=True)


In [170]:
# from tensorflow.keras.optimizers import Adam
# from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, TensorBoard
# !rm -rf ./logs1/

In [171]:
# checkpoint = ModelCheckpoint("basic_model1.h5",
#                              monitor="val_loss",
#                              mode="min",
#                              save_best_only = True,
#                              verbose=1)
# # earlystop= EarlyStopping(monitor = 'val_loss',
# #                             mode="min",
# #                             min_delta = 0,
# #                             patience = 5,
# #                             verbose = 1)

# tensorboard = TensorBoard(log_dir='logs1',histogram_freq=1,write_grads=True)

# callbacks = [checkpoint,tensorboard]

In [4]:
from tensorflow.keras.models import load_model

In [6]:
import tensorflow as tf
model = tf.keras.models.load_model('model.h5')

In [174]:
# # Compile your final_model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
                    loss='categorical_crossentropy',
                    metrics=['accuracy'])

# Compile the model
# final_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
# generator = data_generator(train_dataset)
# final_model.fit(generator, epochs= 15, steps_per_epoch=len(train_dataset), verbose = 1, callbacks = callbacks, validation_data = test_dataset)


# Train your model
h1 = model.fit(train_dataset, epochs=25, verbose=1, validation_data = test_dataset)

#h1 = final_model.fit(train_dataset, epochs=100,
#                      validation_data=test_dataset)


Epoch 1/25
13/13 ━━━━━━━━━━━━━━━━━━━━ 16s 289ms/step - accuracy: 0.3851 - loss: 0.7408 - val_accuracy: 0.3016 - val_loss: 0.7158
Epoch 2/25
13/13 ━━━━━━━━━━━━━━━━━━━━ 3s 208ms/step - accuracy: 0.4671 - loss: 0.6995 - val_accuracy: 0.3333 - val_loss: 0.6988
Epoch 3/25
13/13 ━━━━━━━━━━━━━━━━━━━━ 3s 207ms/step - accuracy: 0.5328 - loss: 0.6919 - val_accuracy: 0.3492 - val_loss: 0.6914
Epoch 4/25
13/13 ━━━━━━━━━━━━━━━━━━━━ 3s 217ms/step - accuracy: 0.5621 - loss: 0.6861 - val_accuracy: 0.5238 - val_loss: 0.6835
Epoch 5/25
13/13 ━━━━━━━━━━━━━━━━━━━━ 3s 223ms/step - accuracy: 0.5955 - loss: 0.6749 - val_accuracy: 0.6667 - val_loss: 0.6687
Epoch 6/25
13/13 ━━━━━━━━━━━━━━━━━━━━ 3s 214ms/step - accuracy: 0.6180 - loss: 0.6589 - val_accuracy: 0.6667 - val_loss: 0.6451
Epoch 7/25
13/13 ━━━━━━━━━━━━━━━━━━━━ 3s 210ms/step - accuracy: 0.6723 - loss: 0.6061 - val_accuracy: 0.6667 - val_loss: 0.6175
Epoch 8/25
13/13 ━━━━━━━━━━━━━━━━━━━━ 3s 210ms/step - accuracy: 0.6731 - loss: 0.5597 - val_accuracy: 0

In [176]:
model.save( 'model_bike.h5')

In [16]:
model = tf.keras.models.load_model('model_bike.h5')

In [ ]:
# for i in range (len(x_test)):
#   tok = Tokenizer()
#   import numpy as np
#   # Evaluate the model on a single row
#   row = x_train.iloc[i]
#   image_features = np.load(str(row['image_id']) + '.npy')
#   print("image_features: ", image_features)
#   print("image id:", row['image_id'])
#   print("question:", row[['question']])
#   tokenized_question = tok.texts_to_sequences([row['question']])
#   tokenized_question = pad_sequences(tokenized_question, maxlen=24)
#   prediction = model.predict([image_features.reshape(1, -1), tokenized_question])
#   print("predicted answer:", ohe.inverse_transform(prediction))
#   print("predicted answer:", prediction)
#   print("actual answer:", y_test.iloc[i])

In [178]:
for i in range (len(x_test)):
  tok = Tokenizer()
  import numpy as np
  # Evaluate the model on a single row
  row = data.iloc[i]
  image_features = np.load(str(row['image_id']) + '.npy')
  tokenized_question = tok.texts_to_sequences([row['question']])
  tokenized_question = pad_sequences(tokenized_question, maxlen=24)
  prediction = model.predict([image_features.reshape(1, -1), tokenized_question])
  print("predicted answer:", ohe.inverse_transform(prediction))
  print("actual answer:", data['answer'][i])

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
predicted answer: [['yes']]
actual answer: yes
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
predicted answer: [['no']]
actual answer: no
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
predicted answer: [['yes']]
actual answer: yes
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
predicted answer: [['yes']]
actual answer: yes
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
predicted answer: [['no']]
actual answer: no
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
predicted answer: [['yes']]
actual answer: yes
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
predicted answer: [['no']]
actual answer: yes
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
predicted answer: [['no']]
actual answer: no
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
predicted answer: [['no']]
actual answer: no
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
predicted answer: [['no']]
actual answer: no
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
predicted answer: [['yes']]
actual answer: yes
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
predicted answer: [['yes']]
actual ans

In [8]:
Img_model = VGG16(weights='imagenet')
feature_model = Model(inputs=Img_model.inputs, outputs=Img_model.layers[-2].output)

In [9]:
import pickle

# Load the pickle file
with open('ohe.pkl', 'rb') as file:
    ohe = pickle.load(file)


In [12]:
import numpy as np
from PIL import Image
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer

# Load the trained model
# model = tf.keras.models.load_model('model.h5')

# Load the tokenizer
tokenizer = Tokenizer()
tokenizer.fit_on_texts(data['question'])

def parse_function(filename):
    image_string = tf.io.read_file(filename)
    #Don't use tf.image.decode_image, or the output shape will be undefined
    image = tf.image.decode_png(image_string, channels=3)
    image = image/255
    #This will convert to float values in [0, 1]
    image = tf.image.convert_image_dtype(image, tf.float32)
    image = tf.image.resize(image, [224, 224])
    image = tf.expand_dims(image, axis=0) 
    return  image, filename

def predict_answer(image_path, question):
    # Load and preprocess the image
    
    # Tokenize and pad the question
    tokenized_question = tokenizer.texts_to_sequences([question])
    tokenized_question = pad_sequences(tokenized_question, maxlen=24)

    # Reshape the image and question for model input
    image,_ = parse_function(image_path)
    image_features = feature_model(image)
    tokenized_question = tokenized_question.reshape(1, -1)

    # Make the prediction
    prediction = model.predict([image_features, tokenized_question])

    # Decode the predicted answer
    predicted_answer = ohe.inverse_transform(prediction)

    return predicted_answer

# Example usage
image_path = 'testImg4.jpeg'
question = 'is the person wearing helmet?'
predicted_answer = predict_answer(image_path, question)
print(predicted_answer)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
[['yes']]


In [17]:
import numpy as np
import gradio as gr
import tensorflow as tf
from PIL import Image
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer

# Load the trained model
# model = tf.keras.models.load_model('model.h5')

# Load the tokenizer
tokenizer = Tokenizer()
tokenizer.fit_on_texts(data['question'])

def parse_function(filename):
    image_string = tf.io.read_file(filename)
    # Don't use tf.image.decode_image, or the output shape will be undefined
    image = tf.image.decode_png(image_string, channels=3)
    image = image/255
    # This will convert to float values in [0, 1]
    image = tf.image.convert_image_dtype(image, tf.float32)
    image = tf.image.resize(image, [224, 224])
    image = tf.expand_dims(image, axis=0) 
    return  image, filename

# def parse_function(image_data):
#     # Decode the image data using TensorFlow
#     image = tf.image.decode_image(image_data, channels=3)
#     # Resize the image
#     image = tf.image.resize(image, [224, 224])
#     # Normalize the pixel values to [0, 1]
#     image = image / 255.0
#     # Add a batch dimension
#     image = tf.expand_dims(image, axis=0)
#     return image, None  # We don't need the file name for prediction




def predict_answer(image, question):
    # Load and preprocess the image
    
    # Tokenize and pad the question
    tokenized_question = tokenizer.texts_to_sequences([question])
    tokenized_question = pad_sequences(tokenized_question, maxlen=24)

    # Reshape the image and question for model input
    image_features, _ = parse_function(image)
    image_features = feature_model(image_features)  # If you have a feature extraction model
    tokenized_question = tokenized_question.reshape(1, -1)

    # Make the prediction
    # prediction = model.predict([image_features, tokenized_question])

    # Decode the predicted answer
    # You will need to replace this with your actual prediction logic
    predicted_answer = model.predict([image_features, tokenized_question])
    predicted_answer = ohe.inverse_transform(predicted_answer)
    predicted_answer = predicted_answer[0][0]
    return predicted_answer

# Gradio Interface
image_input = gr.Image(type='filepath', label="Upload Image")
question_input = gr.Textbox(label="Enter your question here:", placeholder="e.g., Is the person wearing a helmet?")
output = gr.Textbox(label="Prediction")

gr.Interface(fn=predict_answer, inputs=[image_input, question_input], outputs=output, title="VQA Model", description="Predict whether a person is wearing a helmet based on an image and a question.").launch(share = True)
# ans = predict_answer('testImg4.jpeg', 'is the person wearing helmet?')
# print(ans)

Running on local URL:  http://127.0.0.1:7860

Could not create share link. Please check your internet connection or our status page: https://status.gradio.app.


2024/05/01 10:12:23 [W] [service.go:132] login to server failed: dial tcp 44.237.78.176:7000: i/o timeout


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 475ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 169ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step
